In [17]:
from pyspark.sql import SQLContext
from dateutil.parser import parse
from datetime import datetime

def date_and_hour_and_minute(s):
    dt = parse(s.replace('?',' '))
    hour = dt.hour
    minute = dt.minute
    return (dt.strftime("%Y-%m-%d"),hour, minute)

def check_empty_column(row):
    for column in row:
        if column == None:
            return False
    return True

sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load('file:///home/oxclo/BigDataAndCloudComputing/test_bad_data.csv')
yellowcabdata = df.rdd

filteremptycolumn = yellowcabdata.filter(lambda x: check_empty_column(x))
filterzerolatitudelongitude = filteremptycolumn.filter(lambda x: x.pickup_longitude != 0.0 and bool(x.pickup_longitude) and x.pickup_latitude != 0.0 and bool(x.pickup_latitude) and x.dropoff_longitude != 0.0 and bool(x.dropoff_longitude) and x.dropoff_latitude != 0.0 and bool(x.dropoff_latitude))
filterwrongtime = filterzerolatitudelongitude.filter(lambda x: x.tpep_pickup_datetime <= x.tpep_dropoff_datetime)
filterinvaliddistance = filterwrongtime.filter(lambda x: x.trip_distance > 0 or x.payment_type == 6)
filterinvalidfareamount = filterinvaliddistance.filter(lambda x: x.fare_amount > 0 or x.payment_type == 6 or x.payment_type == 3)
filterinvalidfareamount.take(14)
#mapped = filteremptycolumn.map(lambda x: (x.tpep_pickup_datetime, x.tpep_dropoff_datetime, x.trip_distance, x.pickup_longitude, x.pickup_latitude, x.dropoff_longitude, x.dropoff_latitude))
#mapped.take(14)
#mapped = winddata.map(lambda x: (x.Station_ID, date_and_hour(x.Interval_End_Time),x.Wind_Velocity_Mtr_Sec, x.Ambient_Temperature_Deg_C))
#filtered = mapped.filter(lambda (a,b,c,d): c != 0.0 and d != 0.0 and bool(c) and bool(d))
#combinedstationdatetime = filtered.map(lambda (s,(d,h),v,t): ((s,d,h),(v,t)))
#keyvalueone = combinedstationdatetime.map(lambda (station_date_hour,(v,t)): (station_date_hour,(v,t,1)))
#sumdata = keyvalueone.reduceByKey(lambda (v1,t1,c1),(v2,t2,c2):(v1+v2,t1+t2,c1+c2))
#averagedata = sumdata.map(lambda((s,d,h),(v,t,c)):((s,d,h),v/c,t/c))

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2016, 1, 1, 0, 0), tpep_dropoff_datetime=datetime.datetime(2016, 1, 1, 0, 0), passenger_count=2, trip_distance=1.1, pickup_longitude=-73.99037170410156, pickup_latitude=40.73469543457031, RatecodeID=1, store_and_fwd_flag=u'N', dropoff_longitude=-73.98184204101562, dropoff_latitude=40.73240661621094, payment_type=2, fare_amount=7.5, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0, improvement_surcharge=0.3, total_amount=8.8),
 Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2016, 1, 1, 0, 0, 3), tpep_dropoff_datetime=datetime.datetime(2016, 1, 1, 0, 15, 49), passenger_count=6, trip_distance=2.43, pickup_longitude=-73.96932983398438, pickup_latitude=40.7635383605957, RatecodeID=1, store_and_fwd_flag=u'N', dropoff_longitude=-73.99568939208984, dropoff_latitude=40.7442512512207, payment_type=3, fare_amount=0.0, extra=0.5, mta_tax=0.5, tip_amount=3.99, tolls_amount=0, improvement_surcharge=0.3, total_amount=17.29)]